In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

https://stat.gov.pl/obszary-tematyczne/ceny-handel/wskazniki-cen/wskazniki-cen-towarow-i-uslug-konsumpcyjnych-pot-inflacja-/miesieczne-wskazniki-cen-towarow-i-uslug-konsumpcyjnych-od-1982-roku/

In [2]:
### Załadowanie dysku
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
### Załadowanie pliku
data_file_path = "/content/drive/My Drive/Colab Notebooks/miesieczne_wskazniki_cen_towarow_i_uslug_konsumpcyjnych_od_1982_roku.xlsx"
try:
  df = pd.read_excel(data_file_path)
  if "Sposób prezentacji" in df.columns:
      # Filtrowanie danych
      df_poprzedni_miesiac_100 = df[df["Sposób prezentacji"] == "Poprzedni miesiąc = 100"].copy()

      print("Wczytano dane i przefiltrowano według 'Sposób prezentacji' = 'Poprzedni miesiąc=100'.")
  else:
      print("Kolumna 'Sposób prezentacji' nie została znaleziona w pliku Excel.")
except FileNotFoundError:
    print(f"Błąd: Plik nie znaleziony pod ścieżką {data_file_path}")
except Exception as e:
    print(f"Wystąpił błąd podczas wczytywania pliku: {e}")

Wczytano dane i przefiltrowano według 'Sposób prezentacji' = 'Poprzedni miesiąc=100'.


In [4]:
### Filtracja danych
# TODO: sprawdz co oznacza Flaga 'a' w roku 2000 i 2001
#df_poprzedni_miesiac_100[["Sposób prezentacji", "Rok", "Miesiąc", "Wartość", "Flaga"]]
data = df_poprzedni_miesiac_100[["Rok", "Miesiąc", "Wartość"]]
# pozbywamy się wierszy z wartościami NaN, występujące w roku 2025
data = data.dropna(subset=["Wartość"])
data = data.sort_values(by=['Rok','Miesiąc'], ascending=[1,1])

data_array = data.to_numpy()

print(data.head(20))

       Rok  Miesiąc  Wartość
2100  1982        1    106.6
2101  1982        2    136.3
2102  1982        3    105.8
2103  1982        4    103.1
2104  1982        5    102.7
2105  1982        6    100.9
2106  1982        7    103.9
2107  1982        8     99.4
2108  1982        9    100.3
2109  1982       10    101.4
2110  1982       11    100.9
2111  1982       12    100.8
2076  1983        1    101.9
2077  1983        2    107.2
2078  1983        3    101.9
2079  1983        4    103.2
2080  1983        5    101.0
2081  1983        6     99.5
2082  1983        7    100.6
2083  1983        8     99.6


In [5]:
# inclacja - gdy "Wartość" > 100
# deflacja - gdy "Wartość" < 100
# kolumna 0, rok
# kolumna 1, miesiąc
# kolumna 2, wartość względem grudnia 1986
data_len = len(data)
data_array_absolute = np.zeros((len(data), 3))
absolute_result = 1.0
idx = 0
for row in data_array:
  absolute_result = absolute_result + absolute_result * ((float(row[2])-100.0) / 100.0)
  # można zliczać zamiast miesiąca i roku, to zliczać kolejne miesiące od stycznia 1987
  data_array_absolute[idx] = [int(row[0]), int(row[1]), absolute_result]
  idx += 1

In [9]:
# print(data_array_absolute)
X = data_array_absolute[:, 0:2] # daty
y = data_array_absolute[:, 2]   # inflacja
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)